In [14]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import nltk  
import numpy as np  
import random  
import string
import re
from itertools import repeat

In [2]:
df = pd.DataFrame()
df  = pd.read_csv('labels_imperium_all.csv',encoding='utf-8',header=None)
labels = df[0]
df.head()

,0
0,1
1,0
2,0
3,0
4,0


In [3]:
df = pd.DataFrame()
df  = pd.read_csv('imperium_txts_all.csv',encoding='utf-8',header=None)
df.head()

,0
0,"""You fuck your dad."""
1,"""i really don't understand your point.\xa0 It ..."
2,"""A\\xc2\\xa0majority of Canadians can and has ..."
3,"""listen if you dont wanna get married to a man..."
4,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd..."


In [4]:
documents = df[0]
print(documents[0])
# lower case
documents = documents.str.lower() 
# remove punctuation
documents = documents.str.replace('[^\w\s]','') 
documents = documents.values.tolist()
for i in range(len(documents )):
    # tokenize
    documents [i] = nltk.word_tokenize(documents [i])
    # remove blanck spaces
    documents [i] = [re.sub(r'\W',' ',word) for word in documents [i] ]
print("Ther are :",len(documents)," 0:", documents[0])

"You fuck your dad."
Ther are : 8829  0: ['you', 'fuck', 'your', 'dad']


Cleaning the data: Remove puntuation

In [5]:
wordfreq = {}
for sentence in documents:
    for token in sentence:
        if token not in wordfreq.keys():
            wordfreq[token] = 1
        else:
            wordfreq[token] += 1
print("Ther are :",len(wordfreq)," unique words")

Ther are : 30675  unique words


The wordfreq save each unique word and the number of time that his words appears

In [6]:
# Use vectorize function of numpy 
length_checker = np.vectorize(len) 
  
# Find the length of each element 
documents_length = length_checker(documents) 
  
# Print the length of each element 
print(documents_length) 

[  4  15  64 ...   8  52 188]


In [7]:
# List of keys to be deleted from dictionary
selectedKeys = list() 
 
# Iterate over the dict and put to be deleted keys in the list
for (key, value) in wordfreq.items():
    if value < 5:
        selectedKeys.append(key)
    if len(key) < 3:
        selectedKeys.append(key)
    if len(key) > 30:
        selectedKeys.append(key)
 
# Iterate over the list and delete corresponding key from dictionary
for key in selectedKeys:
    if key in wordfreq:
        del wordfreq[key]
print("Ther are :",len(wordfreq)," unique words")

Ther are : 4110  unique words


In [8]:
# Saving each unique word index
word_idx = {}
idx = 0
for token in wordfreq:
    word_idx[token] =  idx
    idx += 1
print(word_idx['florida'])

3020


In [9]:
sentence_vectors = []
for sentence in documents:
    sent_vec = []
    for token in wordfreq:
        if token in sentence:
            df = sentence.count(token)
            sent_vec.append(df)
        else:
            sent_vec.append(0)
    sentence_vectors.append(sent_vec)

In [11]:
sentence_vectors = np.asarray(sentence_vectors)
print("Matrix of dimension: (",sentence_vectors.shape[0],",",sentence_vectors.shape[1],")")

Matrix of dimension: ( 8829 , 4110 )


In [12]:
# Identifying the document with more terms
maxlen_doc = []
for doc in documents:
    maxlen_doc.append(len(doc))
print("Maximum document longitud: ",max(maxlen_doc))  
print(max(documents_length))
maxlen_doc = (max(documents_length))

Maximum document longitud:  2299
2299


Creating an document / index matrix

In [16]:
# We need to separete this in sTrain and Stest, we need to be careful to mantein the index in the training fase
num_docs = sentence_vectors.shape[0]
#maximum = []
idx_matriz = np.zeros((num_docs, maxlen_doc)) # idx_matriz represents "stest" in Matlab
row = 0
for doc in documents:
    # If tern not in dicc, add '0'
    doc_inx = [ word_idx[x] if x in word_idx else 0 for x in doc] 
    doc_inx.extend(repeat(0, maxlen_doc - len(doc_inx)))  # Here we are padding, but it is NOT necessary
    idx_matriz[row] = (doc_inx)
    row += 1
    # Like this ingnore those words tht are not in dic : doc_inx = [wordfreq[x] for x in doc if x in wordfreq]
    #maximum.append(len(doc_inx))
print("Matrix of dimension: (",idx_matriz.shape[0],",",idx_matriz.shape[1],")")

Matrix of dimension: ( 8829 , 2299 )


In [17]:
np.savetxt("document_matrix.csv", sentence_vectors, delimiter=",")

In [18]:
df  = pd.read_csv('document_matrix.csv',encoding='utf-8',header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,4100,4101,4102,4103,4104,4105,4106,4107,4108,4109
0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
X_train = sentence_vectors[:6182]
X_test  = sentence_vectors[6182:]
Y_train = labels[:6182]
Y_test  = labels[6182:]
S_train = idx_matriz[:6182]
S_test  = idx_matriz[6182:]    
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)
print(S_train.shape)
print(S_test.shape)

(6182, 4110)
(6182,)
(2647, 4110)
(2647,)
(6182, 2299)
(2647, 2299)


In [101]:
def csa(X_train,Y,X_test):
    num_terms = X_train.shape[1]
    num_class = 2
    dl_train = documents_length [:6182]
    dl_test  = documents_length [6182:] 
    TR = np.zeros((num_terms, num_class))
    num_docs = X_train.shape[0]
    
    # Non - Normalization 0'
    for j in range (0,num_class): # j = [0;1]
        docs = np.where(Y == j)[0] 
        for i in range(0,num_terms):
            tf_v = X_train[docs,i] # tf the term (v)
            TR[i][j] += sum(np.log2(1 + tf_v / dl_train [docs]))

    #--print("PRB: (",TR.shape[0],",",TR.shape[1],")")
    
    # Normalization 1' Sum all the weights vectors 
    n1 = np.sum(TR, axis = 0) #Sum by colum
    Tik = TR / n1

    # Normalization 2' Each componet of tik [ti1,ti2]/ [ti1+ti2;ti1+ti2]
    for i in range(0,num_terms):
        Tik[i] = Tik[i] / sum(Tik[i]) # Tik[i] = Tik[i] / sum(TR[i])

    #--print("Tik: (",Tik.shape[0],",",Tik.shape[1],")")
    
    # Document Representation
    DR = np.zeros((num_docs, num_class))
    for i in range(0,num_docs):
        termos = np.where(X_train[i] != 0)[0] # extrac idxs =! 0
        a = X_train[i][termos] / dl_train[i]# acces idxs content / len
        a = np.expand_dims(a, axis=1)
        DR[i] = sum(np.multiply(a,Tik[termos]))
        
    # Document Test Representation
    num_docs = X_test.shape[0]
    TDR = np.zeros((num_docs, num_class))
    for i in range(0,num_docs):
        termos = np.where(X_test[i] != 0)[0] # extrac idxs =! 0
        a = X_test[i][termos] / dl_test[i]# acces idxs content / len
        a = np.expand_dims(a, axis=1)
        TDR[i] = sum(np.multiply(a,Tik[termos]))

    return DR,TDR

In [96]:
DR,DRT = csa(X_train,Y_train,X_test)
#print(DRS.shape)
#print(DR_TS.shape)

PRB: ( 4110 , 2 )
Tik: ( 4110 , 2 )


C:\Users\DanielaFe7\Anaconda3\envs\RendesNeurais\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in true_divide


In [100]:
print(DR.shape)
print(DRT.shape)

(6182, 2)
(2647, 2)


In [98]:
len(documents_length)

8829

PBR: u did this for all X, u shoul do this 4 xtrain and x test

In [92]:
"""num_terms = sentence_vectors.shape[1]
num_class = 2
TR = np.zeros((num_terms, num_class))
num_docs = sentence_vectors.shape[0]
"""

'num_terms = sentence_vectors.shape[1]\nnum_class = 2\nTR = np.zeros((num_terms, num_class))\nnum_docs = sentence_vectors.shape[0]\n'

In [99]:
"""
for j in range (0,num_class): # j = [0;1]
        docs = np.where(labels == j)[0] 
        for i in range(0,num_terms):
            tf_v = sentence_vectors[docs,i]
            TR[i][j] += sum(np.log2(1 + tf_v / documents_length [docs]))

print("PRB: (",TR.shape[0],",",TR.shape[1],")")

# Normalization 1' Sum all the weights vectors 
n1 = np.sum(TR, axis = 0) #Sum by colum
Tik = TR / n1

# Normalization 2' Each componet of tik [ti1,ti2]/ [ti1+ti2;ti1+ti2]
for i in range(0,num_terms):
    Tik[i] = Tik[i] / sum(Tik[i]) # Tik[i] = Tik[i] / sum(TR[i])

print("Tik: (",Tik.shape[0],",",Tik.shape[1],")")
"""

'\nfor j in range (0,num_class): # j = [0;1]\n        docs = np.where(labels == j)[0] \n        for i in range(0,num_terms):\n            tf_v = sentence_vectors[docs,i]\n            TR[i][j] += sum(np.log2(1 + tf_v / documents_length [docs]))\n\nprint("PRB: (",TR.shape[0],",",TR.shape[1],")")\n\n# Normalization 1\' Sum all the weights vectors \nn1 = np.sum(TR, axis = 0) #Sum by colum\nTik = TR / n1\n\n# Normalization 2\' Each componet of tik [ti1,ti2]/ [ti1+ti2;ti1+ti2]\nfor i in range(0,num_terms):\n    Tik[i] = Tik[i] / sum(Tik[i]) # Tik[i] = Tik[i] / sum(TR[i])\n\nprint("Tik: (",Tik.shape[0],",",Tik.shape[1],")")\n'

In [93]:
# Document Representation
"""DR = np.zeros((num_docs, num_class))
for i in range(0,num_docs):
    termos = np.where(sentence_vectors[i] != 0)[0] # extrac idxs =! 0
    a = sentence_vectors[i][termos] / documents_length[i]# acces idxs content / len
    a = np.expand_dims(a, axis=1)
    DR[i] = sum(np.multiply(a,Tik[termos]))
DR.shape
# Some of then are empy sentence_vectors[i]  examp 8825 "Nadie se salva de la regla 34 xd"
# none of the owrds bellow to dicc
"""

'DR = np.zeros((num_docs, num_class))\nfor i in range(0,num_docs):\n    termos = np.where(sentence_vectors[i] != 0)[0] # extrac idxs =! 0\n    a = sentence_vectors[i][termos] / documents_length[i]# acces idxs content / len\n    a = np.expand_dims(a, axis=1)\n    DR[i] = sum(np.multiply(a,Tik[termos]))\nDR.shape\n# Some of then are empy sentence_vectors[i]  examp 8825 "Nadie se salva de la regla 34 xd"\n# none of the owrds bellow to dicc\n'

In [89]:
#print("DR: (",DR.shape[0],",",DR.shape[1],")")
#for i in DR:
#    print(i)

In [94]:
"""
X_train = DR[:6182]
X_test  = DR[6182:]
Y_train = labels[:6182]
Y_test  = labels[6182:]
S_train = idx_matriz[:6182]
S_test  = idx_matriz[6182:]
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)
print(S_train.shape)
print(S_test.shape)
"""

'\nX_train = DR[:6182]\nX_test  = DR[6182:]\nY_train = labels[:6182]\nY_test  = labels[6182:]\nS_train = idx_matriz[:6182]\nS_test  = idx_matriz[6182:]\nprint(X_train.shape)\nprint(Y_train.shape)\nprint(X_test.shape)\nprint(Y_test.shape)\nprint(S_train.shape)\nprint(S_test.shape)\n'

In [179]:
#X_train, X_test, Y_train, Y_test = train_test_split( DR, labels, test_size=0.3, random_state=42)
#print(X_train.shape)
#print(Y_train.shape)
#print(X_test.shape)
#print(Y_test.shape)

In [77]:
from keras.models import Sequential
from keras.layers import Dense
import keras
#create model
model = Sequential()
#get number of columns in training data
#add model layers
model.add(Dense(10, activation='relu', input_shape=(num_class,)))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

Using TensorFlow backend.





_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                30        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 41
Trainable params: 41
Non-trainable params: 0
_________________________________________________________________
None


In [79]:
opt = keras.optimizers.SGD(decay=1e-14)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [80]:
model.fit(DR, Y_train, epochs=100, batch_size=10, validation_data=(DRT, Y_test))


Train on 6182 samples, validate on 2647 samples
Epoch 1/100
6182/6182 [==============================] - 2s 398us/step - loss: 0.6357 - acc: 0.6561 - val_loss: nan - val_acc: 0.7261
Epoch 2/100
6182/6182 [==============================] - 1s 158us/step - loss: 0.6065 - acc: 0.6563 - val_loss: nan - val_acc: 0.7269
Epoch 3/100
6182/6182 [==============================] - 1s 158us/step - loss: 0.5814 - acc: 0.6632 - val_loss: nan - val_acc: 0.7416
Epoch 4/100
6182/6182 [==============================] - 1s 158us/step - loss: 0.5533 - acc: 0.6952 - val_loss: nan - val_acc: 0.7469
Epoch 5/100
6182/6182 [==============================] - 1s 158us/step - loss: 0.5225 - acc: 0.7310 - val_loss: nan - val_acc: 0.7544
Epoch 6/100
6182/6182 [==============================] - 1s 159us/step - loss: 0.4913 - acc: 0.7625 - val_loss: nan - val_acc: 0.7703
Epoch 7/100
6182/6182 [==============================] - 1s 158us/step - loss: 0.4615 - acc: 0.7926 - val_loss: nan - val_acc: 0.7752
Epoch 8/100
6

Epoch 60/100
6182/6182 [==============================] - 1s 158us/step - loss: 0.3187 - acc: 0.8530 - val_loss: nan - val_acc: 0.7986
Epoch 61/100
6182/6182 [==============================] - 1s 158us/step - loss: 0.3182 - acc: 0.8539 - val_loss: nan - val_acc: 0.7941
Epoch 62/100
6182/6182 [==============================] - 1s 159us/step - loss: 0.3189 - acc: 0.8526 - val_loss: nan - val_acc: 0.7990
Epoch 63/100
6182/6182 [==============================] - 1s 163us/step - loss: 0.3178 - acc: 0.8509 - val_loss: nan - val_acc: 0.7979
Epoch 64/100
6182/6182 [==============================] - 1s 165us/step - loss: 0.3182 - acc: 0.8551 - val_loss: nan - val_acc: 0.7960
Epoch 65/100
6182/6182 [==============================] - 1s 159us/step - loss: 0.3178 - acc: 0.8536 - val_loss: nan - val_acc: 0.7994
Epoch 66/100
6182/6182 [==============================] - 1s 161us/step - loss: 0.3177 - acc: 0.8528 - val_loss: nan - val_acc: 0.7975
Epoch 67/100
6182/6182 [==============================]

Reading parcial data

In [107]:
retains = [0.0100000000000000,0.0600000000000000,0.110000000000000,0.160000000000000,0.210000000000000,0.260000000000000,0.310000000000000,0.360000000000000,0.410000000000000,0.460000000000000,0.510000000000000,0.560000000000000,0.610000000000000,0.660000000000000,0.710000000000000,0.760000000000000,0.810000000000000,0.860000000000000,0.910000000000000,1]
#retains = [0.910000000000000]

for j in range (0,len(retains)):
    print("---------------------------------------- ",retains[j]," ---------------------------------------------")
    rXtest = np.zeros((X_test.shape[0], num_terms))
    for i in range(0,X_test.shape[0]): # iterating each test document
        noz = np.where(S_test[i] != 0)[0] # from our idx (ONLY INDXS) document matriz, extract idx != 0 (this step could be avoid if we do not agregate those 0)
        ntermssf = round(len(noz)*retains[j]) # We obtain the number of terns with x% of longitud
        myox = 1 # minnum Termos = 1
        wdix = 0 # 
        freqtsof = np.zeros(num_terms)  
        # This vector activate the terms present in the Document until that longitud
        while myox <= ntermssf:
            if S_test[i,wdix] != 0:
                freqtsof[int(S_test[i,wdix])] += 1 #If the same word appears more than once , just add one to the position
                myox += 1
            wdix += 1
        rXtest[i] = freqtsof
        
    DR,DRT = csa(X_train,Y_train,rXtest)
    print(DRS.shape)
    print(DR_TS.shape)
    model.fit(DR, Y_train, epochs=100, batch_size=10, validation_data=(DRT, Y_test))
    #Xtestv = rXtest

----------------------------------------  0.01  ---------------------------------------------


C:\Users\DanielaFe7\Anaconda3\envs\RendesNeurais\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in true_divide


(6182, 2)
(2647, 2)
Train on 6182 samples, validate on 2647 samples
Epoch 1/100
6182/6182 [==============================] - 1s 160us/step - loss: 0.3092 - acc: 0.8586 - val_loss: nan - val_acc: 0.7378
Epoch 2/100
6182/6182 [==============================] - 1s 150us/step - loss: 0.3100 - acc: 0.8570 - val_loss: nan - val_acc: 0.7378
Epoch 3/100
6182/6182 [==============================] - 1s 148us/step - loss: 0.3094 - acc: 0.8585 - val_loss: nan - val_acc: 0.7378
Epoch 4/100
6182/6182 [==============================] - 1s 146us/step - loss: 0.3102 - acc: 0.8596 - val_loss: nan - val_acc: 0.7378
Epoch 5/100
6182/6182 [==============================] - 1s 146us/step - loss: 0.3099 - acc: 0.8593 - val_loss: nan - val_acc: 0.7378
Epoch 6/100
6182/6182 [==============================] - 1s 146us/step - loss: 0.3098 - acc: 0.8578 - val_loss: nan - val_acc: 0.7378
Epoch 7/100
6182/6182 [==============================] - 1s 145us/step - loss: 0.3095 - acc: 0.8581 - val_loss: nan - val_acc: 0

6182/6182 [==============================] - 1s 145us/step - loss: 0.3095 - acc: 0.8580 - val_loss: nan - val_acc: 0.7378
Epoch 62/100
6182/6182 [==============================] - 1s 146us/step - loss: 0.3097 - acc: 0.8578 - val_loss: nan - val_acc: 0.7378
Epoch 63/100
6182/6182 [==============================] - 1s 147us/step - loss: 0.3089 - acc: 0.8606 - val_loss: nan - val_acc: 0.7378
Epoch 64/100
6182/6182 [==============================] - 1s 146us/step - loss: 0.3088 - acc: 0.8606 - val_loss: nan - val_acc: 0.7378
Epoch 65/100
6182/6182 [==============================] - 1s 145us/step - loss: 0.3093 - acc: 0.8601 - val_loss: nan - val_acc: 0.7378
Epoch 66/100
6182/6182 [==============================] - 1s 145us/step - loss: 0.3097 - acc: 0.8598 - val_loss: nan - val_acc: 0.7378
Epoch 67/100
6182/6182 [==============================] - 1s 145us/step - loss: 0.3094 - acc: 0.8598 - val_loss: nan - val_acc: 0.7378
Epoch 68/100
6182/6182 [==============================] - 1s 145us/s

6182/6182 [==============================] - 1s 147us/step - loss: 0.3088 - acc: 0.8615 - val_loss: nan - val_acc: 0.7340
Epoch 21/100
6182/6182 [==============================] - 1s 146us/step - loss: 0.3090 - acc: 0.8610 - val_loss: nan - val_acc: 0.7337
Epoch 22/100
6182/6182 [==============================] - 1s 146us/step - loss: 0.3087 - acc: 0.8607 - val_loss: nan - val_acc: 0.7340
Epoch 23/100
6182/6182 [==============================] - 1s 147us/step - loss: 0.3089 - acc: 0.8630 - val_loss: nan - val_acc: 0.7337
Epoch 24/100
6182/6182 [==============================] - 1s 146us/step - loss: 0.3092 - acc: 0.8604 - val_loss: nan - val_acc: 0.7340
Epoch 25/100
6182/6182 [==============================] - 1s 146us/step - loss: 0.3087 - acc: 0.8589 - val_loss: nan - val_acc: 0.7344
Epoch 26/100
6182/6182 [==============================] - 1s 149us/step - loss: 0.3088 - acc: 0.8601 - val_loss: nan - val_acc: 0.7340
Epoch 27/100
6182/6182 [==============================] - 1s 147us/s

Epoch 81/100
6182/6182 [==============================] - 1s 151us/step - loss: 0.3090 - acc: 0.8627 - val_loss: nan - val_acc: 0.7340
Epoch 82/100
6182/6182 [==============================] - ETA: 0s - loss: 0.3071 - acc: 0.863 - 1s 154us/step - loss: 0.3085 - acc: 0.8628 - val_loss: nan - val_acc: 0.7344
Epoch 83/100
6182/6182 [==============================] - 1s 150us/step - loss: 0.3089 - acc: 0.8604 - val_loss: nan - val_acc: 0.7337
Epoch 84/100
6182/6182 [==============================] - 1s 151us/step - loss: 0.3088 - acc: 0.8610 - val_loss: nan - val_acc: 0.7340
Epoch 85/100
6182/6182 [==============================] - 1s 152us/step - loss: 0.3089 - acc: 0.8615 - val_loss: nan - val_acc: 0.7340
Epoch 86/100
6182/6182 [==============================] - 1s 154us/step - loss: 0.3087 - acc: 0.8623 - val_loss: nan - val_acc: 0.7340
Epoch 87/100
6182/6182 [==============================] - 1s 149us/step - loss: 0.3088 - acc: 0.8586 - val_loss: nan - val_acc: 0.7340
Epoch 88/100
6182

KeyboardInterrupt: 

In [ ]:
hacer funcion las representaciones
verificar el pre procesamietno
PBR: u did this for all X, u shoul do this 4 xtrain and x test